In [ ]:
# main.py
import numpy as np
import pandas as pd
import yfinance as yf
from scipy.optimize import minimize
import matplotlib.pyplot as plt
import sys 

In [ ]:
tickers = ['AAPL', 'MSFT', 'GOOGL', 'AMZN', 'JNJ', 'V']
start_date = '2020-01-01'
end_date = '2024-12-31'

try:
    print("Downloading historical stock data...")
    data = yf.download(tickers, start=start_date, end=end_date)['Close']
    
    if data.empty:
        print("\n--- DOWNLOAD ERROR ---")
        print("No data was downloaded. The DataFrame is empty.")
        print("Please check your ticker symbols, date range, and internet connection.")
        sys.exit() 
        
    missing_tickers = [ticker for ticker in tickers if ticker not in data.columns]
    if missing_tickers:
        print(f"\n--- DOWNLOAD ERROR ---")
        print(f"Could not download data for the following tickers: {missing_tickers}")
        print("Please check if the ticker symbols are correct.")
        sys.exit() 

    if data.isnull().values.any():
        print("\n--- DATA WARNING ---")
        print("Downloaded data contains missing values (NaNs). This can happen if a stock didn't trade on certain days.")
        print("Dropping rows with missing data to proceed...")
        data.dropna(inplace=True)

    print("Data downloaded successfully.")

except Exception as e:
    print(f"\nAn unexpected error occurred during data download: {e}")
    print("Aborting script.")
    sys.exit() 

In [ ]:
log_returns = np.log(data / data.shift(1))
log_returns = log_returns.dropna()

In [ ]:
def get_portfolio_stats(weights, log_returns):

    trading_days = 252
    expected_return = np.sum(log_returns.mean() * weights) * trading_days
    covariance_matrix = log_returns.cov() * trading_days
    expected_volatility = np.sqrt(np.dot(weights.T, np.dot(covariance_matrix, weights)))
    risk_free_rate = 0.02 
    sharpe_ratio = (expected_return - risk_free_rate) / expected_volatility
    return expected_return, expected_volatility, sharpe_ratio

In [ ]:
num_portfolios = 20000
all_weights = np.zeros((num_portfolios, len(tickers)))
ret_arr = np.zeros(num_portfolios)
vol_arr = np.zeros(num_portfolios)
sharpe_arr = np.zeros(num_portfolios)

print("Running Monte Carlo Simulation...")
for i in range(num_portfolios):
    weights = np.random.random(len(tickers))
    weights = weights / np.sum(weights)
    all_weights[i, :] = weights
    portfolio_return, portfolio_volatility, portfolio_sharpe = get_portfolio_stats(weights, log_returns)
    ret_arr[i] = portfolio_return
    vol_arr[i] = portfolio_volatility
    sharpe_arr[i] = portfolio_sharpe

max_sharpe_idx = sharpe_arr.argmax()
max_sharpe_return = ret_arr[max_sharpe_idx]
max_sharpe_vol = vol_arr[max_sharpe_idx]
max_sharpe_weights = all_weights[max_sharpe_idx, :]

min_vol_idx = vol_arr.argmin()
min_vol_return = ret_arr[min_vol_idx]
min_vol_vol = vol_arr[min_vol_idx]
min_vol_weights = all_weights[min_vol_idx, :]

print("\n--- Simulation Results ---")
print("Max Sharpe Ratio Portfolio (from simulation):")
print(f"  - Return: {max_sharpe_return:.2%}")
print(f"  - Volatility: {max_sharpe_vol:.2%}")
print(f"  - Sharpe Ratio: {sharpe_arr.max():.2f}")
print("  - Weights:")
for i, ticker in enumerate(tickers):
    print(f"    {ticker}: {max_sharpe_weights[i]:.2%}")

In [ ]:
def neg_sharpe(weights, log_returns):
    return -get_portfolio_stats(weights, log_returns)[2]

constraints = ({'type': 'eq', 'fun': lambda weights: np.sum(weights) - 1})
bounds = tuple((0, 1) for _ in range(len(tickers)))
initial_guess = np.array(len(tickers) * [1. / len(tickers),])

print("\nRunning Optimizer to find the tangency portfolio (max Sharpe ratio)...")
opt_results = minimize(
    fun=neg_sharpe, 
    x0=initial_guess, 
    args=(log_returns,), 
    method='SLSQP', 
    bounds=bounds, 
    constraints=constraints
)

optimal_weights = opt_results.x
optimal_return, optimal_vol, optimal_sharpe = get_portfolio_stats(optimal_weights, log_returns)

print("\n--- Optimization Results ---")
print("Optimal Portfolio (Max Sharpe Ratio):")
print(f"  - Return: {optimal_return:.2%}")
print(f"  - Volatility: {optimal_vol:.2%}")
print(f"  - Sharpe Ratio: {optimal_sharpe:.2f}")
print("  - Optimal Weights:")
for i, ticker in enumerate(tickers):
    print(f"    {ticker}: {optimal_weights[i]:.2%}")

In [ ]:
plt.style.use('seaborn-v0_8-darkgrid')
plt.figure(figsize=(12, 8))
scatter = plt.scatter(vol_arr, ret_arr, c=sharpe_arr, cmap='viridis', marker='o', s=10, alpha=0.5)
plt.colorbar(scatter, label='Sharpe Ratio')
plt.scatter(max_sharpe_vol, max_sharpe_return, c='orange', s=100, marker='*', edgecolors='black', label='Max Sharpe (Simulation)')
plt.scatter(min_vol_vol, min_vol_return, c='red', s=100, marker='*', edgecolors='black', label='Min Volatility (Simulation)')
plt.scatter(optimal_vol, optimal_return, c='blue', s=150, marker='*', edgecolors='black', label='Optimal Portfolio (Optimizer)')
plt.title('Efficient Frontier & Optimal Portfolio', fontsize=18)
plt.xlabel('Annualized Volatility (Risk)', fontsize=14)
plt.ylabel('Annualized Return', fontsize=14)
plt.legend(loc='upper left', fontsize=12)

from matplotlib.ticker import FuncFormatter
plt.gca().yaxis.set_major_formatter(FuncFormatter(lambda y, _: f'{y:.0%}'))
plt.gca().xaxis.set_major_formatter(FuncFormatter(lambda x, _: f'{x:.0%}'))
plt.show()